In [1]:
# Phishing Website Detection Feature Extraction
# Collecting Data legitimate(0) and phishing(1)

In [2]:
# gerekli olan paket ve kütüphanelerin yüklenmesi
import pandas as pd
# phishing attack URL verilerinin dataframe'e yüklenmesi
data0 = pd.read_csv("online-valid-dataset.csv")
data0.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,6557033,http://u1047531.cp.regruhosting.ru/acces-inges...,http://www.phishtank.com/phish_detail.php?phis...,2020-05-09T22:01:43+00:00,yes,2020-05-09T22:03:07+00:00,yes,Other
1,6557032,http://hoysalacreations.com/wp-content/plugins...,http://www.phishtank.com/phish_detail.php?phis...,2020-05-09T22:01:37+00:00,yes,2020-05-09T22:03:07+00:00,yes,Other
2,6557011,http://www.accsystemprblemhelp.site/checkpoint...,http://www.phishtank.com/phish_detail.php?phis...,2020-05-09T21:54:31+00:00,yes,2020-05-09T21:55:38+00:00,yes,Facebook
3,6557010,http://www.accsystemprblemhelp.site/login_atte...,http://www.phishtank.com/phish_detail.php?phis...,2020-05-09T21:53:48+00:00,yes,2020-05-09T21:54:34+00:00,yes,Facebook
4,6557009,https://firebasestorage.googleapis.com/v0/b/so...,http://www.phishtank.com/phish_detail.php?phis...,2020-05-09T21:49:27+00:00,yes,2020-05-09T21:51:24+00:00,yes,Microsoft


In [3]:
# görüleceği üzere elimizde binlerce phishing attack URL bağlantısı var
# fakat bizim ihtiyacımız olan model eğitimi için belirli bir sayıda veri almak
# veri dengesizliği riskine girmeden 10,000 adet phishing 5,000 adet legitimate alıyoruz
# yukarıdaki dataframe'den rastgele 5000 adet veri çekicez

In [4]:
# rastgele 5,000 adet phishing URL'sini çekiyoruz
phishurl = data0.sample(n = 5000, random_state = 12).copy()
phishurl = phishurl.reset_index(drop=True)
phishurl.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,6514946,http://confirmprofileaccount.com/,http://www.phishtank.com/phish_detail.php?phis...,2020-04-19T11:06:55+00:00,yes,2020-04-19T13:42:41+00:00,yes,Other
1,4927651,http://www.marreme.com/MasterAdmin/04mop.html,http://www.phishtank.com/phish_detail.php?phis...,2017-04-04T19:35:54+00:00,yes,2017-05-03T23:00:42+00:00,yes,Other
2,5116976,http://modsecpaststudents.com/review/,http://www.phishtank.com/phish_detail.php?phis...,2017-07-25T18:48:30+00:00,yes,2017-07-28T16:01:36+00:00,yes,Other
3,6356131,https://docs.google.com/forms/d/e/1FAIpQLScL6L...,http://www.phishtank.com/phish_detail.php?phis...,2020-01-13T20:13:37+00:00,yes,2020-01-17T01:55:38+00:00,yes,Other
4,6535965,https://oportunidadedasemana.com/americanas//?...,http://www.phishtank.com/phish_detail.php?phis...,2020-04-29T00:01:03+00:00,yes,2020-05-01T10:55:35+00:00,yes,Other


In [5]:
# çekilen verilerin kontrol edilmesi
phishurl.shape

(5000, 8)

In [6]:
# phishing olan verilerimizi topladık, şimdi legal (legitimate) olan verileri topluycaz
# legal olan verilerin bulunduğu dosyayı yüklüyoruz, verileri ayıklıyoruz

In [7]:
# legal olan verilerinin dataframe'e yüklenmesi
data1 = pd.read_csv("benign-list-big-final-dataset.csv")
data1.columns = ['URLs']
data1.head()

,URLs
0,http://1337x.to/torrent/1110018/Blackhat-2015-...
1,http://1337x.to/torrent/1122940/Blackhat-2015-...
2,http://1337x.to/torrent/1124395/Fast-and-Furio...
3,http://1337x.to/torrent/1145504/Avengers-Age-o...
4,http://1337x.to/torrent/1160078/Avengers-age-o...


In [8]:
# yukarıdaki dataframe'den 5,000 adet legitimate URL topluyoruz

In [9]:
# rastgele 5,000 adet legitimate URL'sini çekiyoruz
legiurl = data1.sample(n = 5000, random_state = 12).copy()
legiurl = legiurl.reset_index(drop=True)
legiurl.head()

,URLs
0,http://graphicriver.net/search?date=this-month...
1,http://ecnavi.jp/redirect/?url=http://www.cros...
2,https://hubpages.com/signin?explain=follow+Hub...
3,http://extratorrent.cc/torrent/4190536/AOMEI+B...
4,http://icicibank.com/Personal-Banking/offers/o...


In [10]:
# çekilen verilerin kontrol edilmesi
legiurl.shape

(5000, 1)

In [11]:
# FEATURE EXTRACTION: bu aşamada verilerden özelliklerini çıkarıcaz
# çıkarılan özellikleri şu şekilde sınıflandırıcaz;
# - Address Bar Tabanlı Özellikler
# - Domain Alanı Tabanlı Özellikler
# - HTML ve Javascript Tabanlı Özellikler

In [12]:
# Address Bar Tabanlı Özellikler;
# çıkarılabilecek özellikler aşağıda listelenmiştir;
# Domain of URL
# IP Address in URL
# "@" Symbol in URL
# Length of URL
# Depth of URL
# Redirection "//" in URL
# "http/https" in Domain name
# Using URL Shortening Services “TinyURL”
# Prefix or Suffix "-" in Domain
# bu özelliklerin her biri ayıklanırken açıklamasını yapıyor olucaz

In [13]:
# gerekli olan paket ve kütüphanelerin yüklenmesi
from urllib.parse import urlparse,urlencode
import ipaddress
import re

In [14]:
# Domain of URL: burada sadece URL'de bulunan etki alanını (domain) çıkarıyoruz
# bu özelliğin eğitimde pek bir önemi yoktur, hatta modeli eğitirken droplanabilir
def getDomain(url):  
  domain = urlparse(url).netloc
  if re.match(r"^www.",domain):
	       domain = domain.replace("www.","")
  return domain

In [15]:
# IP Address in URL: IP adresinin varlığını kontrol eder
# URL'ler etki alanı (domain) yerine IP adresi bulundurabilir
# domain yerine IP adresi kullanılıyorsa bu birisinin URL aracılığıyla
# verileri çalmaya çalıştığından emin olmamıza sebep olacak bir durumdur
def havingIP(url):
  try:
    ipaddress.ip_address(url)
    ip = 1 # phishing
  except:
    ip = 0 # legitimate
  return ip

In [16]:
# "@" Symbol in URL: burada URL içerisinde @ olup olmadığına bakıyoruz
# URL içerisinde @ simgesinin kullanılması, tarayıcının @ öncesindeki herşeyi
# gözardı ederek @ simgesinden sonraki bağlantıya yönlendirilmesine sebep olur
# bu tarz bağlantılarda genelde sizi yönlendirmek istedikleri asıl bağlantı
# @ simgesinden sonra gelir, bu sayede güvenli bir site aracılığya sizi tuzağa çekerler
def haveAtSign(url):
  if "@" in url:
    at = 1 # phishing    
  else:
    at = 0 # legitimate    
  return at

In [17]:
# Length of URL: bağlantının uzunluğu, URL bağlantısının uzunluğu çok önemlidir
# kimlik avı yapan şahıslar şüpheli kısımları gizlemek için uzun bağlantıları kullanır
# eğer URL'in uzunluğu  54'den büyük veya eşitse; kimlik avı riski mevcuttur
def getLength(url):
  if len(url) < 54:
    length = 0 # legitimate             
  else:
    length = 1 # phishing            
  return length

In [18]:
# Depth of URL: bağlantının derinliği, alt sayfaların sayısını hesaplar
# bağlantı içerisindeki "/" işaretinin sayısına göre URL derinliğini ölçer
def getDepth(url):
  s = urlparse(url).path.split('/')
  depth = 0
  for j in range(len(s)):
    if len(s[j]) != 0:
      depth = depth+1
  return depth

In [19]:
# Redirection "//" in URL: bağlantıda yönlendirme varlığı
# eğer bağlantıda "//" mevcutsa kullanıcı başka bir siteye yönlendirilebilir
# URL'deki "//" işaretinin konumu hesaplanır. eğer URL "http" ile başlıyorsa,
# "//" işaretinin 6. konumda görünmesi gerektiği anlamına geldiğini biliyoruz
# ancak eğer URL "https" ile başlıyorsa "//" işaretinin 7. konumda olması gerekiyor
# protokol sonrası dışında "//" işareti herhangi bir konumdaysa kimlik avı olabilir
def redirection(url):
  pos = url.rfind('//')
  if pos > 6:
    if pos > 7:
      return 1 # phishing  
    else:
      return 0 # legitimate  
  else:
    return 0 # legitimate  

In [20]:
# "http/https" in Domain Name; bağlantıda "http/https" olup olmadığını kontrol eder
# kimlik avcıları kullancıyı tuzağa düşürmek için domain kısmına "https" ekleyebilir
def httpDomain(url):
  domain = urlparse(url).netloc
  if 'https' in domain:
    return 1 # phishing  
  else:
    return 0 # legitimate 

In [21]:
# URL Kısaltma Servisleri Hakkında

In [22]:
shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"

In [23]:
# Using URL Shortening Services “TinyURL”: URL kısaltma servisleri
# internet üzerinde uzun ve karmaşık bir bağlantının ciddi oranda küçültüldüğü,
# ve yerine çok daha kısa bir link sunarak gerekli web sayfasına yönlendirdiği,
# HTTP yönlendirmesi aracılığıyla gerçekleştirilen kimlik avı yöntemidir.
def tinyURL(url):
    match=re.search(shortening_services,url)
    if match:
        return 1 # phishing
    else:
        return 0 # legitimate

In [24]:
# Prefix or Suffix "-" in Domain: bağlantıda "-" varlığını kontrol eder
# Çizgi simgesi legal URL'lerde nadiren kullanılır
# kimlik avı yapanlar, domainde "-" ile ayrılmış ekler kullanarak
# kullanıcıların legal bir web sayfasına girdiklerini düşünmesini sağlar
def prefixSuffix(url):
    if '-' in urlparse(url).netloc:
        return 1 # phishing
    else:
        return 0 # legitimate

In [25]:
# Domain Tabanlı Özellikler;
# bu kategoride birçok özellik çıkarılabilir
# biz en önemli olanları dikkate alıyor olucaz
# çıkarılabilecek özellikler aşağıda listelenmiştir;
# - DNS Record
# - Website Traffic
# - Age of Domain
# - End Period of Domain
# bu özelliklerin her biri ayıklanırken açıklamasını yapıyor olucaz

In [26]:
# gerekli olan paket ve kütüphanelerin yüklenmesi
!pip install python-whois

In [27]:
# gerekli olan paket ve kütüphanelerin yüklenmesi
import re
from bs4 import BeautifulSoup
import whois
import urllib
import urllib.request
from datetime import datetime

In [28]:
# DNS Record: phishing yapılan sitelerde kimliğin WHOIS veritabanında bulunmaması
# kimlik avı yapılan web sitelerinde iddia edilen kimliğin WHOIS veritabanı üzerinde
# yapılan kontrolde çıkmaması, kimliğin tanınmıyor olması veya ana makine adı için
# hiçbir kayıp oluşturulmamış olması durumudur. DNS kaydının boş olması veyahut
# bulunamıyor olması durumunda phishing attack söz konusu olabilir

# NOT: "DNS Record" işlemi "featureExtraction" fonksiyonu içinde yapılmaktadır!

In [29]:
# Web Traffic: ziyaretçi sayısını ve ziyaret edilen sayfa sayısını ölçer
# web sitesinin ziyaret sayısı ve ziyaret durumunu kullanarak popülerliğini ölçer,
# kimlik avında kullanılan birçoğu website kısa bir süreliğine ayakta kaldığı için
# bu tarz siteler Alexa (Web Information Company) veri tabanında tanınmayabilirler.
# Ayrıca domain trafiği yoksa veya Alexa'da tanınmıyorsa phishing attack olabilir

def web_traffic(url):
    try:
        # URL'yi kodlayarak geçersiz karakterleri kaldırın
        encoded_url = urllib.parse.quote(url, safe=':/')
        response = urllib.request.urlopen(f"http://data.alexa.com/data?cli=10&dat=s&url={encoded_url}", timeout=10)
        soup = BeautifulSoup(response.read(), "xml")
        reach = soup.find("REACH")
        
        # REACH bulunamadığında rank'e 1 ata
        if reach is None:
            return 1
        
        rank = int(reach['RANK'])
        
    except urllib.error.URLError as e:
        print(f"URL error: {e.reason}")
        return 1
    except Exception as e:
        print(f"General error: {str(e)}")
        return 1
    
    # Rank kontrolü
    if rank < 100000:
        return 1 # phishing
    else:
        return 0 # legitimate

# Test URL
test_url = "http://www.example.com"
print(web_traffic(test_url))

# veri setimizi temin ettiğimiz kullanıcılar bu veri setinin sahip olduğu URL'ler hakkında
# en kötü senaryolarda bile legal web sitelerinin ilk 100,000'de yer aldığını belirttiği için
# biz de onların kullandığı yöntemlere göre hareket ediyoruz.

URL error: [Errno 11001] getaddrinfo failed
1


In [30]:
# Age of Domain: domain adının yaşını tespit eder
# bu özellik de DNS Record ile benzer şekilde WHOIS veri tabanından çıkarılabilir
# kimlik avı yapılan web sitelerin çoğu kısa bir süre ayakta kalır diye söylemiştik
# yaptığımız araştırmalar sonucunda alan adı yaşını min. 6 ay tercih ettik
# buradaki süre, oluşturulma ve son bulma arasında geçen zamandır
def domainAge(domain_name):
  creation_date = domain_name.creation_date
  expiration_date = domain_name.expiration_date
  if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
    try:
      creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except:
      return 1
  if ((expiration_date is None) or (creation_date is None)):
      return 1
  elif ((type(expiration_date) is list) or (type(creation_date) is list)):
      return 1
  else:
    ageofdomain = abs((expiration_date - creation_date).days)
    if ((ageofdomain/30) < 6):
      age = 1 # phishing
    else:
      age = 0 # legitimate
  return age

In [31]:
# End Period of Domain: alan adının bitiş süresini tespit eder
# yine bu özellik de WHOIS veri tabanından çıkarılabilir
# domain'in son kullanım süresini hesaplar (anlık saat ile son kullanma tarihi arası)
# yaptığımız araştırmalarda legal sitelerin alan adı bitiş süresini 6 aydan düşük tespit ettik
# eğer bitiş süresi için 6 ay veya daha fazla bir süre kaldıysa phishing attack olabilir
def domainEnd(domain_name):
  expiration_date = domain_name.expiration_date
  if isinstance(expiration_date,str):
    try:
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except:
      return 1
  if (expiration_date is None):
      return 1
  elif (type(expiration_date) is list):
      return 1
  else:
    today = datetime.now()
    end = abs((expiration_date - today).days)
    if ((end/30) < 6):
      end = 0 # legitimate
    else:
      end = 1 # phishing
  return end

In [32]:
# HTML ve Javascript Tabanlı Özellikler;
# bu kategoride birçok özellik çıkarılabilir
# biz en önemli olanları dikkate alıyor olucaz
# çıkarılabilecek özellikler aşağıda listelenmiştir;
# - IFrame Redirection
# - Status Bar Customization
# - Disabling Right Click
# - Website Forwarding
# bu özelliklerin her biri ayıklanırken açıklamasını yapıyor olucaz

In [33]:
# gerekli olan paket ve kütüphanelerin yüklenmesi
import requests

In [34]:
# IFrame Redirection: ek bir sayfayı görüntülemek için kullanılan HTML etiketi
# kimlik avı yapanlar "iframe" etiketini kullanabilir ve onu görünmez hale getirebilir
# bu bağlamda kimlik avı yapanlar tarayıcının görsel bir tanımlama yapmasına
# imkan sağlayan "frameBorder" özelliğini kullanıyorlar
def iframe(response):
  if response == "":
      return 1
  else:
      if re.findall(r"[<iframe>|<frameBorder>]", response.text):
          return 0 # legitimate
      else:
          return 1 # phishing

In [35]:
# Status Bar Customization: durum çubuğunda sahte URL göstermek
# kimlik avı yapanlar kullanıcıları kandırmak için bu yöntemi tercih eder
# bu yöntemi uygularken JavaScript kullanırlar
# bu özelliği çıkarmak için web sitesinin kaynak kodunu, özellikle "onMouseOver"
# etkinliğinin durum çubuğunda bir değişiklik yapıp yapmadığını kontrol etmeliyiz
def mouseOver(response): 
  if response == "" :
    return 1
  else:
    if re.findall("<script>.+onmouseover.+</script>", response.text):
      return 1 # phishing
    else:
      return 0 # legitimate

In [36]:
# Disabling Right Click: sağ tıklamayı devre dışı bırakmak
# kimlik avı yapanlar, kullanıcının web sayfasının kaynak kodunu görüntüleyememesi
# veya kaydedememesi için JavaScript kullanarak sağ tıklama işlemini devre dışı bırakır
# Bu özellik tam olarak "using onMouseOver to hide the Link" olarak değerlendirilir
# Ancak bu özellik için web sitesinin kaynak kodunda "event.button==2" etkinliğini
# kontrol edicez ve sağ tıklamanın devre dışı bırakılıp bırakılmadığına bakıcaz
def rightClick(response):
  if response == "":
    return 1
  else:
    if re.findall(r"event.button ?== ?2", response.text):
      return 0 # legitimate
    else:
      return 1 # phishing

In [37]:
# Website Forwarding: web sitesinin kaç kez yönlendirildiği
# legal web siteleri genellikle en fazla bir kez yönlendirilmektedir
# kimlik avı için kullanılan sitelerse ise bu sayı ortalama 4 kezdir
# yaptığımız araştırmalar bize bu sonucu verdi (kullanılan veri setine göre değişebilir)
def forwarding(response):
  if response == "":
    return 1
  else:
    if len(response.history) <= 2:
      return 0 # legitimate
    else:
      return 1 # phishing

In [38]:
# COMPUTING URL FEATURES: Özelliklerin toplanması
# tüm özellikleri listede saklayan bir fonksiyon oluşturucaz
# her URL için bu özellikleri çıkarıp bu listeye ekliycez

In [39]:
# özellik çıkarma fonksiyonu
def featureExtraction(url,label):

  features = []
  # Address Bar tabanlı özellikler (10)
  features.append(getDomain(url))
  features.append(havingIP(url))
  features.append(haveAtSign(url))
  features.append(getLength(url))
  features.append(getDepth(url))
  features.append(redirection(url))
  features.append(httpDomain(url))
  features.append(tinyURL(url))
  features.append(prefixSuffix(url))
  
  # Domain tabanlı özellikler  (4)
  dns = 0
  try:
    domain_name = whois.whois(urlparse(url).netloc)
  except:
    dns = 1

  features.append(dns)
  features.append(web_traffic(url))
  features.append(1 if dns == 1 else domainAge(domain_name))
  features.append(1 if dns == 1 else domainEnd(domain_name))
  
  # HTML & Javascript tabanlı özellikler (4)
  try:
    response = requests.get(url)
  except:
    response = ""
  features.append(iframe(response))
  features.append(mouseOver(response))
  features.append(rightClick(response))
  features.append(forwarding(response))
  features.append(label)
  
  return features

In [44]:
# Legitimate URL (legal bağlantılar) için özellik çıkarma işlemi
legiurl.shape

(5000, 1)

In [ ]:
# özelliklerin çıkarılması ve listede saklanması
legi_features = [] 
label = 0

for i in range(0, 5000):
  url = legiurl['URLs'][i]
  legi_features.append(featureExtraction(url,label))

In [ ]:
# listeye dataframe'e dönüştürüyoruz
feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection', 
                      'https_Domain', 'TinyURL', 'Prefix/Suffix', 'DNS_Record', 'Web_Traffic', 
                      'Domain_Age', 'Domain_End', 'iFrame', 'Mouse_Over','Right_Click', 'Web_Forwards', 'Label']

legitimate = pd.DataFrame(legi_features, columns= feature_names)
legitimate.head()